# Train Custom Ocean Trash Detector (YOLOv8) - TACO Edition

This notebook trains a custom object detection model using **YOLOv8** (Ultralytics) on the **TACO (Trash Annotations in Context)** dataset.
This dataset is excellent for marine debris and includes classes like **Wood**, **Shoe**, **Plastic**, **Metal**, etc.

**Steps:**
1. Install YOLOv8
2. Download TACO Dataset (YOLO format)
3. Train Model
4. Export to `.tflite`

In [ ]:
# 0. NUCLEAR CLEANUP
# Run this to clear everything if you get errors
import os
import shutil

# Delete all files in current directory to ensure fresh start
!rm -rf *
print("Cleaned up environment.")

In [ ]:
# 1. Install Dependencies
!pip install ultralytics roboflow
from ultralytics import YOLO
from IPython.display import display, Image

In [ ]:
# 2. Download Dataset
from roboflow import Roboflow

rf = Roboflow(api_key="pkq5npOyduclLgniEcgt")
# User selected dataset: tom-rowland-03ams/taco-wikc7
project = rf.workspace("tom-rowland-03ams").project("taco-wikc7")
version = project.version(3)

# Download in standard YOLOv8 format (not OBB, for TFLite compatibility)
print("Downloading dataset...")
dataset = version.download("yolov8")

# Set the dataset path
dataset_path = dataset.location
print(f"Dataset downloaded to: {dataset_path}")

In [ ]:
# 3. Train Model
# Load a model
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)

# Train the model
# We use the data.yaml file from the downloaded dataset
results = model.train(data=f'{dataset_path}/data.yaml', epochs=20, imgsz=640)

In [ ]:
# 4. Export to TFLite
# Export the model to TFLite format
model.export(format='tflite')

# The exported model is usually saved in runs/detect/train/weights/best_saved_model/best_float32.tflite
# Let's find it and rename it
import glob
# Find the latest run directory
latest_run = max(glob.glob('runs/detect/train*'), key=os.path.getmtime)
tflite_path = f'{latest_run}/weights/best_saved_model/best_float32.tflite'

print(f"Found TFLite model at: {tflite_path}")
!cp {tflite_path} model.tflite

In [ ]:
# 5. Create Labels File
# YOLO stores class names in the model.names attribute
with open('labels.txt', 'w') as f:
    for name in model.names.values():
        f.write(name + '\n')

print("Labels exported to labels.txt")

In [ ]:
# 6. Download Files
from google.colab import files
files.download('model.tflite')
files.download('labels.txt')